<a href="https://colab.research.google.com/github/sochachai/Transformer_Analysis/blob/main/Transformer_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load packages

In [13]:
# check directory
!pwd

/content


In [15]:
# confirm the assistant py file has been uploaded to the correct directory
ls

my_transformer_utils.py  sample_data/


In [1]:
# check Python version
!python --version

Python 3.10.12


In [2]:
# install torch
!pip install torch==2.2.0

In [3]:
# check torch version
import torch
print(torch.__version__)

2.2.0+cu121


In [6]:
import copy
import torch
import math
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
# Issue caused by torch version described below:
# the original package should be pyitcast.transformer_utils
# the problem is that pyitcast.transformer_utils relies on an old version of torch (1.3.1 should work)
# and causes the error "IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python..."
# but the installation of an old version of torch that match pyitcast.transformer is not trivial
# versions of 1.11.0 or after is not compatible with pyitcast.transformer and installation of them will not
# solve the issue
# To solve this issue:
# download the pyitcast.transformer_utils (open by clicking the error message) as a py file
# modify the last line of the SimpleLossCompute class from "return loss.data[0] * norm"
# to "return loss.data * norm"

from my_transformer_utils import Batch
from my_transformer_utils import run_epoch
from my_transformer_utils import greedy_decode
from my_transformer_utils import get_std_opt # get_std_opt is based on Adam optimizer
from my_transformer_utils import LabelSmoothing # offset human label errors to prevent overfitting
from my_transformer_utils import SimpleLossCompute # calculate loss after smoothing, use cross_entropy_loss


Construct the class of Embeddings and Positional Encoding.

In [7]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        '''
        :param d_model: embedding dimension
        :param vocab: size of vocabulary
        '''
        # Initialization
        super(Embeddings, self).__init__()
        # Defrine a word embedding object
        self.lut = nn.Embedding(vocab, d_model)
        # Instantiate d_model
        self.d_model = d_model

    def forward(self, x):
        '''
        :param x: tensor representing the original text
        '''
        return self.lut(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len = 5000):
        '''
        :param d_model: dimension of the encoding
        :param dropout: dropout rate from 0 to 1
        :param max_len: the maximum length of a sentence
        '''
        # Inherit the initialization of nn.Module
        super(PositionalEncoding, self).__init__()

        # Objectify dropout
        self.dropout = nn.Dropout(p=dropout)

        # Inherit a positional encoder matrix, max_len * d_model
        pe = torch.zeros(max_len, d_model)

        # Inherit an absolute position matrix, max_len * 1
        position = torch.arange(0, max_len).unsqueeze(1)

        # Define the conversion matrix, initialization with gap = 2
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0)/d_model))

        # Copy the absolute position matrix to the positional encoder matrix
        # by sine and cosine wave according to the parity of column indices
        pe[:, 0::2] = torch.sin(position * div_term) # even indiced columns are imputed by sine
        pe[:, 1::2] = torch.cos(position * div_term) # odd indiced columns are imputed by cosine

        # Extend pe to 3-dimensional tensor
        pe = pe.unsqueeze(0)

        # Register pe to a buffer, the buffer is not a parameter of the class
        # the buffer will not be updated along with the model update
        # but it can be loaded along with the model
        self.register_buffer('pe', pe)

    def forward(self, x):
        '''
        :param x: Tensor of text
        :return: x + the positional encoding
        '''
        # Shrink the size of pe to save storage
        # by converting the second dimension, i.e. the dimension of max_len
        # to the size of the sentence len of x, i.e. the second dimension of x
        x = x + Variable(self.pe[:,:x.size(1)], requires_grad = False) # False: pe will not be updated
        return self.dropout(x)

def attention(query, key, value, mask = None, dropout = None):
    '''
    :param query: vectorized original text
    :param key: key words of text
    :param value: the original value of key, summarization of query
    :param mask: hide words to avoid data leakage
    :param dropout: dropout rate of neural network
    :return:
    '''
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9) # compare each position with 0

    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)

    return torch.matmul(p_attn, value), p_attn

def clones(module, N):
    '''
    :param module: one attention layer
    :param N: the number of module
    '''
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)]) # deepcopy uses a different memory

class MultiHeadedAttention(nn.Module):
    def __init__(self, head, embedding_dim, dropout = 0.1):
        '''
        :param head: the number of heads
        :param embedding_dim: the embedding dimension
        :param dropout: default dropout rate set to 0.1
        '''
        # Inherit the initialization
        super(MultiHeadedAttention, self).__init__()
        # head must be an integral factor of embedding_dim
        assert embedding_dim % head == 0
        # each head is assigned with the following dimension
        self.d_k = embedding_dim // head # division in the integral domain Z
        # substantiate head
        self.head = head
        # create linear layers, we need 4 of them for Q, K, V and the final connection
        self.linears = clones(nn.Linear(embedding_dim, embedding_dim), 4)
        # create the attention and dropout rate
        self.attn = None
        self.dropout = nn.Dropout(p = dropout)

    def forward(self, query, key, value, mask = None):
        if mask is not None:
            mask = mask.unsqueeze(1)
        batch_size = query.size(0)

        # We have 4 linears and only zipping the first 3 with Q, K, V, the last linear is not used.
        # view(batch_size, -1, self.head, self.d_k).transpose(1,2)
        # is not the same with view(batch_size, self.head, -1, self.d_k)
        # self.head and self.d_k should be neighboring to get embedding_dim in order for the tensor
        # to interpret the relationship between the meaning of words of their positions in a sentence

        query, key, value = \
            [model(x).view(batch_size, -1, self.head, self.d_k).transpose(1,2)
             for model, x in zip(self.linears, (query, key, value))]

        x, self.attn = attention(query, key, value, mask = mask, dropout = self.dropout)

        # Reshape x
        # must use contiguous method after the transpose before the view method
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.head * self.d_k)

        # Pass x to the 4th linear layer
        return self.linears[-1](x)

class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        '''
        :param d_model: embedding dimension
        :param d_ff: transitional dimension
        :param dropout: default dropout set to 0.1
        '''
        super(PositionwiseFeedForward, self).__init__()
        self.w1 = nn.Linear(d_model, d_ff)
        self.w2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w2(self.dropout(F.relu(self.w1(x))))

class LayerNorm(nn.Module):
    def __init__(self, features, eps = 1e-6):
        '''
        :param features: embedding dimension
        :param eps: avoid zero denominator
        '''
        super(LayerNorm, self).__init__()
        # nn.Parameter formats the variables to intrinsic parameters
        # they will be updated along with the model in contrast with buffer
        self.a2 = nn.Parameter(torch.ones(features))
        self.b2 = nn.Parameter(torch.zeros(features))
        # initialization of eps
        self.eps = eps

    def forward(self, x):
        '''
        :param x: the output of previous layer
        :return: numerical standardized x
        '''
        mean = x.mean(-1, keepdim = True)
        std = x.std(-1, keepdim = True)
        return self.a2 * (x - mean) / (std + self.eps) + self.b2

class SublayerConnection(nn.Module):
    def __init__(self, size, dropout = 0.1):
        '''
        :param size: embedding size
        :param dropout: deactivation of neurons to avoid overfitting
        '''
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, sublayer):
        '''
        :param x: the output of previous layer
        :param sublayer: a function, e.g. Multihead_Attention, PositionwiseFeedForward etc.
        :return: x plus sublayer functioning on normed x with dropout
        '''
        return x + self.dropout(sublayer(self.norm(x)))

class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        '''
        :param size: embedding dimension
        :param self_attn: attention
        :param feed_forward: positionwise feed forward
        :param dropout: avoid overfitting
        '''
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        '''
        :param x: output of previous layer
        :param mask: tensor mask to prevent data leakage
        '''
        # input matrix followed by operating function, returns an output matrix
        # sublayer(x, function) will return function(x)
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask)) # the forward function of multihead attention
        return self.sublayer[1](x, self.feed_forward) # the forward function of pointwise feedforward

class Encoder(nn.Module):
    # Encoder is a collection of Encoder Layers
    def __init__(self, layer, N):
        '''
        :param layer: one encoder layer
        :param N: the number of encoder layers
        '''
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        for layer in self.layers: x = layer(x, mask)
        return self.norm(x)

class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        '''
        :param size: embedding dimension
        :param self_attn: masked multihead attention
        :param src_attn: multihead attention
        :param feed_forward: pointwise feed forward
        :param dropout: avoid overfitting
        '''
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        # 3 sublayer for a decoder layer
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, source_mask, target_mask):
        '''
        :param x: output of previous layer
        :param memory: result of encoder
        :param source_mask: delete unnecessary info to improve model performance
        :param target_mask: hide info to prevent data leakage
        :return: output tensor
        '''
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, target_mask))
        # for second layer, Q = x, K = V = m
        x = self.sublayer[1](x, lambda x: self.self_attn(x, m, m, source_mask))
        return self.sublayer[2](x, self.feed_forward)

class Decoder(nn.Module):
    # Decoder is a collection of Decoder Layers
    def __init__(self, layer, N):
        '''
        :param layer: decoder layer
        :param N: the number of decoder layers
        '''
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, source_mask, target_mask):
        for layer in self.layers:
            x = layer(x, memory, source_mask, target_mask)
        return self.norm(x)

class Generator(nn.Module):
    def __init__(self, d_model, vocab_size):
        '''
        :param d_model: embedding dimension
        :param vocab_size: the size of the vocabulary
        '''
        super(Generator, self).__init__()
        self.project = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        return F.log_softmax(self.project(x), dim = -1)

class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, source_embed, target_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = source_embed
        self.tgt_embed = target_embed
        self.generator = generator

    def forward(self, source, target, source_mask, target_mask):
        # encoded source used as memory in decode function
        return self.decode(self.encode(source, source_mask), source_mask,
                           target, target_mask)

    def encode(self, source, source_mask):
        return self.encoder(self.src_embed(source), source_mask)

    def decode(self, memory, source_mask, target, target_mask):
        # embedded target as x in the decoder function
        return self.decoder(self.tgt_embed(target), memory, source_mask, target_mask)

def make_model(source_vocab, target_vocab, N=6,\
               d_model=512, d_ff=2048, head=8, dropout=0.1):
    c = copy.deepcopy
    attn = MultiHeadedAttention(head, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn),c(ff), dropout), N),
        # note the order of vocab_size and d_model;
        # nn.Embedding is not the same with Embeddings;
        # check Embedding_Encoder.py for more;
        nn.Sequential(Embeddings(d_model, source_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, target_vocab), c(position)),
        Generator(d_model, target_vocab)
    )
    for p in model.parameters():
        if p.dim() > 1: nn.init.xavier_uniform_(p) # initialization: make p uniformly sampled; check xavier_uniform for details
    return model



'''
Task: Test the language model's capability to copy a sequence of numbers
      Input a list of numbers, the output should be the identical list of numbers
SubTask: Write a data generator to generate sample data for model testing
'''

# use a function to generate data
def data_generator(V, batch_size, num_batch):
    '''
    :param V: the maximal data value + 1
    :param batch_size: sample data size of one round of training after which model parameters are updated
    :param num_batch: number of rounds of training
    '''
    for i in range(num_batch):
        # data value from 1 to V-1, with data matrix shape = batch_size times 10
        data = torch.from_numpy(np.random.randint(1, V, size = (batch_size, 10)))

        # set starting position label
        data[:, 0] = 1

        # for a copy task source data and target data should be the same
        # no gradient calculation involved
        source = Variable(data, requires_grad = False)
        target = Variable(data, requires_grad = False)
        yield Batch(source, target)


def run(model, loss, epochs=10):
    '''
    :param model: model
    :param loss: loss function
    :param epochs: number of rounds of training
    '''
    for epoch in range(epochs):
        # train model, update model parameters
        model.train()
        run_epoch(data_generator(V, 20, 8), model, loss)
        # evaluate model, no parameters update
        model.eval()
        run_epoch(data_generator(V, 5, 8), model, loss)


Instantiate an example to demonstrate the training of the model. Commented contents are code to check the functionalities of components.

In [8]:
# Instantiate variables
V = 11
batch_size = 20
num_batch = 30

# get model
model = make_model(V, V, N = 2)

# get optimizer
model_optimizer = get_std_opt(model)

# get smooth criterion
criterion = LabelSmoothing(size = V, padding_idx = 0, smoothing = 0.0)

# get loss function
# model as an EncoderDecoder whose last element is a generator object
loss = SimpleLossCompute(model.generator, criterion, model_optimizer)
if __name__ == '__main__':
    run(model, loss)

'''
# demonstration of xavier_uniform
w = torch.empty(3, 5)
w = nn.init.xavier_uniform_(w, gain = nn.init.calculate_gain('relu'))
print(w)

# input parameters
source_vocab = 11
target_vocab = 11
N = 6

# application
if __name__ == '__main__':
    res = make_model(source_vocab, target_vocab, N)
    print(res)
    print(res.generator)
'''

'''
d_model = 512
vocab = 1000
dropout = 0.1
max_len = 60

original_text = Variable(torch.LongTensor([[132,8,521,308],[491,398,999,223]]))
emb = Embeddings(d_model, vocab)
embr = emb(original_text)
#print(f"The resulting tensor after Embeddings:\n {embr}")

# continue with embr
x= embr
pe = PositionalEncoding(d_model, dropout, max_len)
pe_result = pe(x)
#print(f"The resulting tensor after positional encoding:\n {pe_result}")
#print(f"The shape of pe_result: {pe_result.shape}")

# continue with pe_result
size = 512
head = 8
d_model = 512
d_ff = 64
dropout = 0.2
c = copy.deepcopy
attn = MultiHeadedAttention(head, d_model)
ff = PositionwiseFeedForward(d_model, d_ff, dropout)
layer = EncoderLayer(size, c(attn), c(ff), dropout)
N = 8
mask = Variable(torch.zeros(2, 4, 4))

# continue with en_result
en = Encoder(layer, N)
en_result = en(x, mask)
layer = DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout)
N = 8
memory = en_result
# In practice source_mask and target_mask are not the same
mask = Variable(torch.zeros(2, 4, 4))
source_mask = target_mask = mask

# get decoder result
de = Decoder(layer, N)
de_result = de(x, memory, source_mask, target_mask)
#print(de_result)
#print(de_result.shape)


# application
vocab_size = 1000
d_model= 512
gen = Generator(d_model, vocab_size)
source_embed = nn.Embedding(vocab_size, d_model)
target_embed = nn.Embedding(vocab_size, d_model)
# input parameters (assuming source and target are the same)
source = target = Variable(torch.LongTensor([[100, 2, 421, 500], [491, 998, 1, 221]]))
source_mask = target_mask = Variable(torch.zeros(2, 4, 4))

ed = EncoderDecoder(en, de, source_embed, target_embed, gen)
ed_result = ed(source, target, source_mask, target_mask)
print(ed_result)
print(ed_result.shape)
'''

/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch Step: 1 Loss: 3.279118 Tokens per Sec: 372.940277
Epoch Step: 1 Loss: 3.188324 Tokens per Sec: 238.977463
Epoch Step: 1 Loss: 3.076892 Tokens per Sec: 419.207275
Epoch Step: 1 Loss: 2.559503 Tokens per Sec: 166.714081
Epoch Step: 1 Loss: 2.650356 Tokens per Sec: 418.882477
Epoch Step: 1 Loss: 2.144870 Tokens per Sec: 247.651245
Epoch Step: 1 Loss: 2.441571 Tokens per Sec: 443.532562
Epoch Step: 1 Loss: 2.110667 Tokens per Sec: 249.359802
Epoch Step: 1 Loss: 2.138038 Tokens per Sec: 292.783783
Epoch Step: 1 Loss: 1.910373 Tokens per Sec: 247.901703
Epoch Step: 1 Loss: 2.024286 Tokens per Sec: 419.112701
Epoch Step: 1 Loss: 1.892929 Tokens per Sec: 242.006073
Epoch Step: 1 Loss: 2.044051 Tokens per Sec: 435.513428
Epoch Step: 1 Loss: 1.915974 Tokens per Sec: 187.736664
Epoch Step: 1 Loss: 1.988847 Tokens per Sec: 442.701874
Epoch Step: 1 Loss: 1.682120 Tokens per Sec: 253.315094
Epoch Step: 1 Loss: 1.928860 Tokens per Sec: 346.934662
Epoch Step: 1 Loss: 1.551572 Tokens per Sec: 192

'\nd_model = 512\nvocab = 1000\ndropout = 0.1\nmax_len = 60\n\noriginal_text = Variable(torch.LongTensor([[132,8,521,308],[491,398,999,223]]))\nemb = Embeddings(d_model, vocab)\nembr = emb(original_text)\n#print(f"The resulting tensor after Embeddings:\n {embr}")\n\n# continue with embr\nx= embr\npe = PositionalEncoding(d_model, dropout, max_len)\npe_result = pe(x)\n#print(f"The resulting tensor after positional encoding:\n {pe_result}")\n#print(f"The shape of pe_result: {pe_result.shape}")\n\n# continue with pe_result\nsize = 512\nhead = 8\nd_model = 512\nd_ff = 64\ndropout = 0.2\nc = copy.deepcopy\nattn = MultiHeadedAttention(head, d_model)\nff = PositionwiseFeedForward(d_model, d_ff, dropout)\nlayer = EncoderLayer(size, c(attn), c(ff), dropout)\nN = 8\nmask = Variable(torch.zeros(2, 4, 4))\n\n# continue with en_result\nen = Encoder(layer, N)\nen_result = en(x, mask)\nlayer = DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout)\nN = 8\nmemory = en_result\n# In practice source_mask